In [3]:
using LinearAlgebra
using Tables
using DataFrames
using CSV
using Catan
using CatanLearning

[ Info: Adding new configs: /home/bryan/Projects/Catan/CatanLearning.jl/src/../DefaultConfiguration.toml


In [4]:
features_csv = "/home/bryan/Projects/Catan/CatanLearning.jl/data/features.csv"
df = CatanLearning.load_typed_features_from_csv(features_csv)

Row,CountSettlement,CountTotalSettlement,CountCity,CountRoad,MaxRoadLength,SumWoodDiceWeight,SumBrickDiceWeight,SumPastureDiceWeight,SumStoneDiceWeight,SumGrainDiceWeight,CountPortWood,CountPortBrick,CountPortPasture,CountPortStone,CountPortGrain,CountTotalWood,CountTotalBrick,CountTotalPasture,CountTotalStone,CountTotalGrain,CountDevCardsKnight,CountDevCardsMonopoly,CountDevCardsYearOfPlenty,CountDevCardsRoadBuilding,CountDevCardsVictoryPoint,HasLargestArmy,HasLongestRoad,WonGame
,Int8,Int8,Int8,Int8,Int8,Int16,Int16,Int16,Int16,Int16,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Bool,Bool,Cat…
1,2,2,0,3,2,6,0,2,0,11,0,0,0,0,0,5,5,6,0,8,0,0,0,0,0,false,false,false
2,2,2,0,2,1,0,4,5,5,0,0,0,0,0,0,6,6,9,7,2,0,0,0,0,0,false,false,false
3,2,2,0,2,1,0,11,2,5,3,0,0,0,0,0,10,10,6,4,3,0,0,0,0,0,false,false,false
4,2,2,0,2,1,4,0,0,0,5,0,0,0,0,0,4,4,2,0,7,0,0,0,0,0,false,false,false
5,2,2,0,3,2,0,4,0,0,5,0,0,0,0,0,5,5,2,0,2,0,0,0,0,0,false,false,false
6,2,2,0,5,3,6,4,2,1,3,0,0,0,0,0,11,11,2,1,3,0,0,0,0,0,false,false,false
7,2,2,0,5,2,2,4,3,0,3,0,0,0,0,0,8,8,6,0,3,0,0,0,0,0,false,false,false
8,2,2,0,2,1,6,0,0,0,8,0,1,0,0,0,6,6,2,0,4,0,0,0,0,0,false,false,false
9,1,2,1,2,1,2,0,0,10,8,0,0,0,0,0,3,3,5,7,11,1,0,0,0,0,false,false,false


In [5]:
y = [value == true ? 1.0 : 0.0 for value in df[:, :WonGame]]
# Extract the feature matrix
X = Matrix(df[:, Not(:WonGame)])
(m,n) = size(X)
U,S,Vt = svd(X)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
211169×27 Matrix{Float64}:
 -0.00208837   0.00148831    0.00091066   …   0.990486      0.000431657
 -0.00200507   8.56047e-5    0.00131806      -0.136996      0.00375413
 -0.00253196  -0.00180983   -0.00158925       9.73024e-6   -0.000858054
 -0.00137719   0.000634233  -0.000260201     -9.85019e-6    0.000102426
 -0.00129344  -0.000940281  -0.000861422     -6.59877e-6   -0.000621633
 -0.00239747  -0.00237692   -0.00137789   …  -7.37721e-6   -0.00236759
 -0.00204396  -0.00177761    0.000809794     -2.17254e-6   -0.00130077
 -0.00162872   0.000126431  -0.000730498     -3.3506e-5    -0.00177491
 -0.00208532   0.00429178   -0.00161082      -2.82881e-5   -0.00299166
 -0.00191771   0.00236043    0.000530097     -2.77781e-5    0.00240059
  ⋮                                       ⋱   ⋮            
 -0.00779474  -0.00510145   -0.00576713   …   9.52362e-5    0.00621907
 -0.00826403  -0.00176363   -0.00802073       0.000174087   0.

In [6]:
threshold = 0.01
N_num = length(filter(x -> x >= threshold, S))


25

In [7]:
pseudo_inv = transpose(Vt)[1:n, 1:N_num] * diagm(S[1:N_num]) * transpose(U)[1:N_num, 1:m]
model = pseudo_inv * y

27-element Vector{Float64}:
   8.196609170146388
   1.7562240830648355
  -6.253038288141411
   6.7163427602920684
 -15.416532508509901
   9.579051502912318
  -4.791003406272694
  36.46632925864589
   0.1973846122887263
  22.917205681466392
   ⋮
   5.175219914373906
  -0.37732618199977846
   1.0219443020708452
   5.916744119160844
  -2.875731226771853
   4.0614565378771115
   0.810001888084425
  75.21185983146667
   0.3410094626086074

In [8]:
df = DataFrame(Weights = model)
CSV.write("../../models/linear_model.csv", df)

"../../models/linear_model.csv"

In [ ]:
function train_model(features_csv):
    df = CatanLearning.load_typed_features_from_csv(features_csv)
    y = [value == true ? 1.0 : 0.0 for value in df[:, :WonGame]]
    # Extract the feature matrix
    X = Matrix(df[:, Not(:WonGame)])
    (m,n) = size(X)
    U,S,Vt = svd(X)
    threshold = 0.01
    N_num = length(filter(x -> x >= threshold, S))
    pseudo_inv = transpose(Vt)[1:n, 1:N_num] * diagm(S[1:N_num]) * transpose(U)[1:N_num, 1:m]
    model = pseudo_inv * y
    return model

In [ ]:
features_csv = "/home/bryan/Projects/Catan/CatanLearning.jl/data/features.csv"

train_model(features_csv)